In [140]:
import numpy as np
from fractions import Fraction
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
from sympy.core.assumptions import assumptions
from sympy import linprog

In [117]:
OUTCOME_PROB = Fraction(1,6)

In [162]:
x, y, z = symbols('x y z')
#assumptions(x)

In [163]:
V = [x>0, x<y, y<z]

def no_loser(a,b,c):
    return a>0 & b>0 & c>0

In [164]:
# The assumptions framework in sympy does not work so well with linear inequalities
# We use our own LP framework to detect:
# 1. constraints that violate V
# 2. inferences (e.g. 0<x<y and 3*x < y => 2*x < y)
from sympy.solvers.simplex import lpmax
from sympy.core import *

EPS = 1e-65
def to_nonstrict_ineq(ineq):
    if isinstance(ineq, Gt):
        # a>b => a >= b+EPS
        return Ge(ineq.lhs, ineq.rhs + EPS)
    elif isinstance(ineq, Lt):
        # a<b => a <= b-EPS
        return Le(ineq.lhs, ineq.rhs - EPS)
    else:
        return ineq  # already non-strict

In [177]:
from sympy.solvers.simplex import InfeasibleLPError

def solve_lp(e, ineqs, f=lpmax):
    # convert all constraints to non-strict inequalities
    print(ineqs)
    constraints = [to_nonstrict_ineq(c) for c in ineqs]
    print(constraints)
    return f(e, constraints)

def feasible(ineq, assumptions):
    try:
        result = solve_lp(0, assumptions + [ineq])
    except InfeasibleLPError:
        return False
    return result

def redundant(ineq, assumptions):
    # if Assumptions => Ineq (<=> not A or I) is valid, then (A and not I) should be unsatisfiable
    return feasible(Not(ineq), assumptions) == False

[x > 0, x < y, x + y <= 0]
[x >= 1.0e-65, x <= y - 1.0e-65, x + y <= 0]


True

In [139]:
import sympy as sp

def is_inferred(ineq, assumptions):
    # Convert inequality and assumptions from strings to sympy expressions
    ineq_expr = sp.sympify(ineq)
    assumption_exprs = [sp.sympify(assumption) for assumption in assumptions]
    
    # Negate the inequality to check for contradictions
    negated_ineq_expr = sp.Not(ineq_expr)

    # Combine the assumptions with the negation of the inequality
    combined_expr = sp.And(*assumption_exprs, negated_ineq_expr)
    
    # Check if the combined expression leads to a contradiction (is False)
    result = sp.simplify(combined_expr)
    return result is sp.false

# Examples
print(is_inferred('y > 0', ['x > 0', 'x < y']))  # True
print(is_inferred('x - y < 0', ['x > 0', 'x < y']))  # True
print(is_inferred('x < 2*y', ['x > 0', 'x < y']))  # True


True
True
False


In [122]:
def getParentStates(a,b,c):
    return [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)]

In [123]:
getParentStates(y, 2*x, z-x)

[(2*y, 2*x - y, -x + z),
 (2*y, 2*x, -x - y + z),
 (-2*x + y, 4*x, -x + z),
 (y, 4*x, -3*x + z),
 (x + y - z, 2*x, -2*x + 2*z),
 (y, 3*x - z, -2*x + 2*z)]

In [124]:
# base case
def h1(a,b,c):
    return OUTCOME_PROB * And(a>0,b>0,c>0) * ((a<=b) + (a<=c))

In [125]:
import collections
import functools

class memoized(object):
    '''Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    '''
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [181]:
def reduce(ineqs, assumptions):
    # converts the inferred inequalities under `assumptions` to 0s or 1s
    reduced = []
    const = 0
    for ineq in ineqs:
        if not feasible(ineq, assumptions):
            # not feasible
            continue
        elif redundant(ineq, assumptions):
            # definitely true under assumptions
            const += 1
        else:
            reduced.append(ineq)
    return {'c': const, 'ind': reduced}

In [182]:
@memoized
def _h(n,a,b,c):
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return [simplify(a<=b), simplify(a<=c)]
    # list of inequalities representing the region Rn
    return list(set(sum([_h(n-1, *s) for s in getParentStates(a,b,c)], [])))

@memoized
def h(n,a,b,c):
    return reduce(_h(n,a,b,c), V)

def f(n,a,b,c,R,S,V=V):
    region = And(*R, *S, *V)
    # convert all indicators inferrable from [region] to 1s -> redundant constraint
    # convert all exclusive indicators to [region] to 0 -> infeasible lp
    # rest: + terms to 0, - terms to 1 (want lb)
    
    return

class H():
    def __init__(self,n,a,b,c):
        self.coef = pow(OUTCOME_PROB, n)
        self.regions = h(n,a,b,c)
        print(self.regions)

In [183]:
h(3,x,y,z)

[x > 0, x < y, y < z, z >= 2*x]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, z >= 2*x]
[x > 0, x < y, y < z, z < 2*x]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, z <= 2*x - 1.0e-65]
[x > 0, x < y, y < z, 3*x <= y + 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 3*x <= y + 3*z]
[x > 0, x < y, y < z, 3*x > y + 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 3*x >= y + 3*z + 1.0e-65]
[x > 0, x < y, y < z, 4*x <= y]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 4*x <= y]
[x > 0, x < y, y < z, 4*x > y]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 4*x >= y + 1.0e-65]
[x > 0, x < y, y < z, x <= -2*y + 2*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= -2*y + 2*z]
[x > 0, x < y, y < z, x > -2*y + 2*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x >= -2*y + 2*z + 1.0e-65]
[x > 0, x < y, y < z, x <= y + 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= y + 3*z]
[x > 0, x < y, y < z, x > y + 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65,

[x > 0, x < y, y < z, x <= 6*y]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= 6*y]
[x > 0, x < y, y < z, x > 6*y]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x >= 6*y + 1.0e-65]
[x > 0, x < y, y < z, x <= 2*y + z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= 2*y + z]
[x > 0, x < y, y < z, x > 2*y + z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x >= 2*y + z + 1.0e-65]
[x > 0, x < y, y < z, 2*x <= 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 2*x <= 3*z]
[x > 0, x < y, y < z, 2*x > 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, 2*x >= 3*z + 1.0e-65]
[x > 0, x < y, y < z, x <= y - z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= y - z]
[x > 0, x < y, y < z, x <= 6*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= 6*z]
[x > 0, x < y, y < z, x > 6*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x >= 6*z + 1.0e-65]
[x > 0, x < y, y < z, x <= 3*y - 3*z]
[x >= 1.0e-65, x <= y - 1.0e-65, y <= z - 1.0e-65, x <= 3*y -

{'c': 26,
 'ind': [z >= 2*x,
  4*x <= y,
  x <= -2*y + 2*z,
  z >= 7*x,
  y >= 6*x,
  3*x <= 2*y,
  3*x <= 2*z,
  3*x <= -y + z,
  x <= -y + z,
  3*x <= y + z,
  4*x <= z,
  x <= 3*y - z,
  y >= 5*x,
  2*x <= -y + z,
  z >= 6*x,
  y >= 2*x,
  5*x <= 3*y,
  x <= -3*y + 3*z,
  5*x <= 3*z,
  x <= -3*y + z,
  z >= 5*x,
  y >= 7*x]}